In [85]:
from dreye.api.optimize import lsq_linear as lsq
import numpy as np
import cvxpy as cp

In [86]:
n, m = 4, 6
A = np.random.random((n, m))
xtarget = np.ones(m)/2
#np.random.random(m)
qtarget = A @ xtarget
# qtarget = np.random.random((1, m))
etarget = qtarget / (1+qtarget)

In [87]:
x = cp.Variable((m), pos=True)

In [88]:
q = A @ x

In [89]:
e = q / (1+q)

In [122]:
e

Expression(QUASILINEAR, NONNEGATIVE, (4,))

In [131]:
cp.sum(q)/cp.sum(1+q)  # intensity

Expression(QUASILINEAR, NONNEGATIVE, ())

In [135]:
c = cp.Variable(1, pos=True)

In [161]:
q/cp.sum(q)

Expression(QUASILINEAR, NONNEGATIVE, (4,))

In [163]:
q == cp.sum(q) * qtarget/np.sum(qtarget)

Equality(Expression(AFFINE, NONNEGATIVE, (4,)), Expression(AFFINE, NONNEGATIVE, (4,)))

In [159]:
testc = cp.sum(q) == np.sum(qtarget)/np.sum(1+qtarget) * cp.sum(1+q)

In [160]:
testc.is_dqcp()

True

In [91]:
denom = cp.multiply((1+q), (1+qtarget))

In [92]:
num = q - qtarget

In [93]:
cp.sum(cp.abs(num) / denom)

Expression(UNKNOWN, NONNEGATIVE, ())

In [94]:
obj = cp.Minimize(cp.max(cp.abs(num)/denom))

In [95]:
constraints = [
    x <= 1
]

In [109]:
obj = cp.Minimize(cp.sum_squares(x - cp.sum(x)/x.size))
obj

Minimize(Expression(CONVEX, NONNEGATIVE, ()))

In [110]:
constraints = [
    x <= 1, 
    cp.max(cp.abs(num)/denom) <= 1e-4
]

In [111]:
prob = cp.Problem(obj, constraints)

In [112]:
prob

Problem(Minimize(Expression(CONVEX, NONNEGATIVE, ())), [Inequality(Variable((6,), nonneg=True, pos=True)), Inequality(Expression(QUASICONVEX, NONNEGATIVE, ()))])

In [113]:
prob.solve(qcp=True, verbose=True, solver=cp.SCS)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Nov 08 06:50:02 PM: Your problem has 6 variables, 2 constraints, and 0 parameters.
(CVXPY) Nov 08 06:50:02 PM: It is compliant with the following grammars: DQCP
(CVXPY) Nov 08 06:50:02 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 08 06:50:02 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 08 06:50:02 PM: Reducing DQCP problem to a one-parameter family of DCP problems, for bisection.

********************************************************************************
Preparing to bisect problem

minimize 0.0
subject to var3673 <= 1.0
           var17732 <= Promote(0.0001, (4,)) @ (Promote(1.0, (4,)) + [[0.80947276 0.98737422 0.54474422 0.76392688 0.5409652  0.13349494]
 [0.42873786 0

1.896216909907639e-08

In [114]:
np.round(x.value, 2)

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [115]:
xtarget

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])